In [1]:
import numpy as np
import math
import torch
from string import punctuation
import torch.nn as nn
from ray import tune
import os

## Reading text files for reviews and sentiments

In [2]:
with open('data/reviews.txt') as f:
    reviews=f.read()
with open('data/labels.txt') as f:
    labels=f.read()

In [3]:
# make all words lowercase
reviews=reviews.lower()

#create reviews without punctuation and split it by new row
all_text=''.join(c for c in reviews if c not in punctuation)
all_text=all_text.split('\n')
final=''.join(all_text)

# make list of all words in text
final=final.split()


We have 6 milion words in text.

In [4]:
len(final)

6020196

In [5]:
final[:6]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy']

## Tokenize words

Tokenize every word which appear in text to int value

In [6]:
unique_words=set(final)
pairs={w:i for (i,w) in enumerate(unique_words)}


## Tokenize every review

In [7]:
tokenized=[]
for review in all_text:
    tokenized.append([pairs[i] for (w,i) in enumerate(review.split())])
tokenized[:3]

[[21588,
  34929,
  71345,
  54376,
  53116,
  45001,
  616,
  19645,
  9477,
  56308,
  53180,
  68148,
  14528,
  28088,
  17168,
  11183,
  55873,
  494,
  18515,
  31601,
  14528,
  11170,
  58918,
  54970,
  6927,
  56308,
  8037,
  68528,
  54275,
  27694,
  45582,
  32807,
  8772,
  21588,
  34929,
  73604,
  5932,
  71345,
  19324,
  63720,
  45582,
  74045,
  43423,
  71345,
  11170,
  56308,
  32559,
  45582,
  40383,
  51895,
  56308,
  24072,
  55075,
  22239,
  28959,
  12331,
  69372,
  72782,
  65265,
  39700,
  11170,
  51076,
  56308,
  15051,
  43816,
  56308,
  59239,
  22611,
  24124,
  70261,
  27694,
  43816,
  56308,
  58460,
  36249,
  61371,
  58131,
  65265,
  55075,
  55069,
  36249,
  34720,
  56308,
  65138,
  6927,
  15582,
  54376,
  37068,
  70126,
  3558,
  45582,
  2464,
  56892,
  56308,
  494,
  36249,
  42228,
  68594,
  9477,
  34929,
  54376,
  52116,
  18143,
  57198,
  36249,
  16007,
  47800,
  45582,
  15867,
  15486,
  43816,
  64196,
  11170

In [8]:
labels_split=labels.split('\n')
encoded_labels=np.array([1 if l=='positive' else 0 for l in labels_split])

## Discard those reviews with zero length

In [9]:
from collections import Counter
lengths=Counter([len(x) for x in tokenized])
lengths[0]

1

In [10]:
for (i,t) in enumerate(tokenized):
    if(len(t)==0):
        del tokenized[i]
        encoded_labels=np.delete(encoded_labels,i)

In [11]:
lengths=Counter([len(x) for x in tokenized])
lengths[0]


0

## Clipping every review to array of fixed size
Clip array to fixed size length or if length of array is lower than fixed length for the input to LSTM, then fill the beginning of the array with zeros.

In [12]:
def fixed_size_features(tokenized,length):
    features=np.zeros((len(tokenized),length),dtype=int)
    for i,t in enumerate(tokenized):
        features[i,-len(t):]=np.array(t)[:length]
    return features


In [13]:
features=fixed_size_features(tokenized,200)

## Packing features to training, validation and test set

In [14]:
split_frac=0.8
idx=np.arange(len(features))
np.random.shuffle(idx)

train_len=int((split_frac*len(features)))
val_len=int(((1-split_frac)/2*len(features)))
train_x=features[idx[:train_len],:]
train_y=encoded_labels[idx[:train_len]]
val_x=features[idx[train_len:train_len+val_len+1],:]
val_y=encoded_labels[idx[train_len:train_len+val_len+1]]
test_x=features[idx[train_len+val_len+1:],:]
test_y=encoded_labels[idx[train_len+val_len+1:]]

In [15]:
from torch.utils.data import DataLoader,TensorDataset

trainset=TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
testset=TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))
valset=TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))

batch_size=50

trainloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader=DataLoader(testset,batch_size=batch_size,shuffle=True)
valloader=DataLoader(valset,batch_size=batch_size,shuffle=True)

In [16]:
# obtain one batch of training data
dataiter = iter(trainloader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ..., 46661,   883, 29805],
        [    0,     0,     0,  ..., 31972, 66889, 72990],
        [11397, 27555, 71345,  ..., 13739, 52546, 66430],
        ...,
        [    0,     0,     0,  ..., 66377, 50180, 14972],
        [    0,     0,     0,  ..., 37918, 54376, 58602],
        [    0,     0,     0,  ..., 63788, 40007, 40007]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1], dtype=torch.int32)


## Defining model

First we need to convert our words to embeddings which are input to LSTM. Every word in sentence is input to different LSTM and hidden layer is sent from previous LSTM to current LSTM as information about history. Also we use Softmax activation function to predict probability if the sentiment of the sentence. So, we are applying softmax to output of the last LSTM and that is the probability that sentence is positive or negative. 

In [17]:
import torch.nn as nn
class SentimentRNN(nn.Module):
    def __init__(self,vocab_size,hidden_size,output_size,n_layers,embedding_size,dropout_p=0.5):
        super(SentimentRNN,self).__init__()
        self.n_layers=n_layers
        self.hidden_size=hidden_size
        self.embedding=nn.Embedding(vocab_size,embedding_size)
        self.lstm=nn.LSTM(embedding_size,hidden_size,n_layers,batch_first=True,dropout=dropout_p)
        self.dropout=nn.Dropout(0.3)
        
        self.fc=nn.Linear(hidden_size,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size=x.size(0)
        x=self.embedding(x.long())
        r_out,hidden=self.lstm(x,hidden)
        
        r_out=r_out.contiguous().view(-1,self.hidden_size)
        out=self.fc(self.dropout(r_out))
        out=self.sigmoid(out)
        out=out.view(batch_size,-1)
        out=out[:,-1]
        return out,hidden
    
    def init_hidden(self,batch_size):
        weights = next(self.parameters()).data
        hidden=(weights.new(self.n_layers,batch_size,self.hidden_size).zero_(),
                      weights.new(self.n_layers,batch_size,self.hidden_size).zero_())
        return hidden
        

## Here is presented structure of our model

vocab_size is how many different words we have in our dataset

since we are predicting is the sentence positive or negative we have one output

n_layers is set to 2 which means that we have 2 stacked LSTMs

In [18]:
# Instantiate the model w/ hyperparams
vocab_size = len(unique_words)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_size = 400
hidden_size=256
n_layers = 2

## Training the model

Since we have to predict 2 classes, we are using Binary CrossEntropy Loss, here I use Adam optimizer with different learning rates which are written in config. I am using ray for hyperparameter tuning.

I tuned LSTM hyperparameters and also training hyperparameters.

In [19]:
def train_net(config,checkpoint_dir=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = SentimentRNN(vocab_size,hidden_size,output_size,n_layers,embedding_size,dropout_p=config["dropout"])

    net.cuda()
    # training params
    epochs = config["epochs"] # 3-4 is approx where I noticed the validation loss stop decreasing

    counter = 0
    print_every = 100
    clip=5 # gradient clipping
    criterion=nn.BCELoss()
    optimizer=torch.optim.Adam(net.parameters(),lr=config["lr"])

    net.train()
    min_loss=math.inf
    min_val_acc=math.inf
    # train for some number of epochs
    for e in range(epochs):
        print(e)
        # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels in trainloader:
            counter += 1
            inputs,labels=inputs.cuda(),labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                val_acc=[]
                corrects=0
                net.eval()
                for inputs, labels in valloader:
                    inputs,labels=inputs.cuda(),labels.cuda()
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, labels.float())
                    pred=torch.round(output)
                    correct_tensor = pred.eq(labels.float().view_as(pred))
                    correct = np.squeeze(correct_tensor.cpu().numpy()) 
                    corrects+=np.sum(correct)
                    val_losses.append(val_loss.item())
                    
                if(np.mean(val_losses)<min_loss):
                    min_loss=np.mean(val_losses)
                    min_val_acc=corrects/len(valloader.dataset)
                    {"min_loss":min_loss}
#                     torch.save(net.state_dict(), 'model.pt')
                    with tune.checkpoint_dir(epochs) as checkpoint_dir:
                        path = os.path.join(checkpoint_dir, "checkpoint")
                        torch.save((net.state_dict(), optimizer.state_dict()), path)
                tune.report(val_acc=min_val_acc,val_loss=min_loss)
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [20]:
config={
    "lr":tune.grid_search([0.001,0.01,0.1]),#
    "epochs":tune.grid_search([3,5]),
    "dropout":tune.grid_search([0.5]),
}

In [21]:
analysis=tune.run(
    train_net,
    config=config,
    resources_per_trial={'gpu': 1}
    )

2020-12-05 20:45:55,877	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-05 20:46:05,509	ERROR syncer.py:63 -- Log sync requires rsync to be installed.
2020-12-05 20:46:05,858	WARNING worker.py:1091 -- Warning: The actor ImplicitFunc has size 18097937 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,dropout,epochs,lr
train_net_83017_00000,RUNNING,,0.5,3,0.001


(pid=8592) 0
Result for train_net_83017_00000:
  date: 2020-12-05_20-46-25
  done: false
  experiment_id: 076ef2c575d548d2bf386704622cdb4d
  experiment_tag: 0_dropout=0.5,epochs=3,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 1
  node_ip: 192.168.0.20
  pid: 8592
  should_checkpoint: true
  time_since_restore: 16.681294679641724
  time_this_iter_s: 16.681294679641724
  time_total_s: 16.681294679641724
  timestamp: 1607197585
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '83017_00000'
  val_acc: 0.6416
  val_loss: 0.633596373796463
  


Trial name,status,loc,dropout,epochs,lr,iter,total time (s),val_acc,val_loss
train_net_83017_00000,RUNNING,192.168.0.20:8592,0.5,3,0.001,1,16.6813,0.6416,0.633596
train_net_83017_00001,PENDING,,0.5,5,0.001,,,,


(pid=8592) Epoch: 1/3... Step: 100... Loss: 0.650763... Val Loss: 0.633596
Result for train_net_83017_00000:
  date: 2020-12-05_20-46-37
  done: false
  experiment_id: 076ef2c575d548d2bf386704622cdb4d
  experiment_tag: 0_dropout=0.5,epochs=3,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 2
  node_ip: 192.168.0.20
  pid: 8592
  should_checkpoint: true
  time_since_restore: 28.997989177703857
  time_this_iter_s: 12.316694498062134
  time_total_s: 28.997989177703857
  timestamp: 1607197597
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: '83017_00000'
  val_acc: 0.6908
  val_loss: 0.5957593601942063
  


...

(pid=13408) Epoch: 5/5... Step: 1900... Loss: 0.694481... Val Loss: 0.693073
Result for train_net_83017_00005:
  date: 2020-12-05_21-05-17
  done: false
  experiment_id: a69247b02f714f479be5d9dd89bb6dd2
  experiment_tag: 5_dropout=0.5,epochs=5,lr=0.1
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 20
  node_ip: 192.168.0.20
  pid: 13408
  time_since_restore: 232.56262588500977
  time_this_iter_s: 10.926835060119629
  time_total_s: 232.56262588500977
  timestamp: 1607198717
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: '83017_00005'
  val_acc: 0.5036
  val_loss: 0.6930728399753571
  


Trial name,status,loc,dropout,epochs,lr,iter,total time (s),val_acc,val_loss
train_net_83017_00005,RUNNING,192.168.0.20:13408,0.5,5,0.1,20,232.563,0.5036,0.693073
train_net_83017_00000,TERMINATED,,0.5,3,0.001,12,150.568,0.8224,0.39639
train_net_83017_00001,TERMINATED,,0.5,5,0.001,20,233.692,0.8268,0.399471
train_net_83017_00002,TERMINATED,,0.5,3,0.01,12,143.941,0.8308,0.392363
train_net_83017_00003,TERMINATED,,0.5,5,0.01,20,233.191,0.8264,0.423312
train_net_83017_00004,TERMINATED,,0.5,3,0.1,12,138.904,0.4976,0.693437


Trial name,status,loc,dropout,epochs,lr,iter,total time (s),val_acc,val_loss
train_net_83017_00000,TERMINATED,,0.5,3,0.001,12,150.568,0.8224,0.39639
train_net_83017_00001,TERMINATED,,0.5,5,0.001,20,233.692,0.8268,0.399471
train_net_83017_00002,TERMINATED,,0.5,3,0.01,12,143.941,0.8308,0.392363
train_net_83017_00003,TERMINATED,,0.5,5,0.01,20,233.191,0.8264,0.423312
train_net_83017_00004,TERMINATED,,0.5,3,0.1,12,138.904,0.4976,0.693437
train_net_83017_00005,TERMINATED,,0.5,5,0.1,20,232.563,0.5036,0.693073


2020-12-05 21:05:17,693	INFO tune.py:439 -- Total run time: 1162.59 seconds (1152.59 seconds for the tuning loop).


(pid=13408) Epoch: 5/5... Step: 2000... Loss: 0.836307... Val Loss: 0.859200


Getting config with best minimum mean loss on validation set

In [22]:
best_conf=analysis.get_best_config(metric="val_loss", mode="min")

In [23]:
best_conf

{'lr': 0.01, 'epochs': 3, 'dropout': 0.5}

## Loading saved network with optimal parameters

Making constructor for network with optimal parameters and then loding best saved checkpoint for that model.

In [24]:
best_trial = analysis.get_best_trial("val_loss", "min", "last")
best_trained_model = (best_trial.config["lr"])    
best_trained_model=SentimentRNN(vocab_size,hidden_size,output_size,n_layers,embedding_size)#

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(
    best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)


<All keys matched successfully>

## Testing the network

In [25]:
def test(model):

    # Get test data loss and accuracy

    test_losses = [] # track loss
    num_correct = 0
    net=model.cuda()
    criterion=nn.BCELoss()
    optimizer=torch.optim.Adam(net.parameters(),lr=best_trial.config["lr"])
    # init hidden state
    h = net.init_hidden(batch_size)

    net.eval()
    # iterate over test data
    for inputs, labels in testloader:
        inputs,labels=inputs.cuda(),labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # get predicted outputs
        output, h = net(inputs, h)

        # calculate loss
        test_loss = criterion(output.squeeze(), labels.float())
        test_losses.append(test_loss.item())

        # convert output probabilities to predicted class (0 or 1)
        pred = torch.round(output.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy()) 
        num_correct += np.sum(correct)


    # -- stats! -- ##
    # avg test loss
    print("Test loss: {:.3f}".format(np.mean(test_losses)))

    # accuracy over all test data
    test_acc = num_correct/len(testloader.dataset)
    print("Test accuracy: {:.3f}".format(test_acc))

In [26]:
test(best_trained_model)

Test loss: 0.394
Test accuracy: 0.836


## We are taking our example of review and testing it on LSTM

First we need to preprocess our sentence, tokenize it, convert to Tensor and then forward it to the input of the LSTM to make sentiment prediction of the sentence.

In [27]:
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [28]:
def prepare_for_network(text, sequence_length):
    text=text.lower()
    text_without_punctuation=''.join([c for c in text if c not in punctuation])
    test_words=text_without_punctuation.split()
    test_ints=[]
    test_ints.append([pairs[w] for w in test_words])
    test_input=fixed_size_features(test_ints,sequence_length)
    return test_input

In [29]:
inputs=prepare_for_network(test_review_pos,200)

In [30]:
inputs

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [31]:
def predict_sentiment(net,sequence):
    seq=torch.tensor(sequence)
    batch_size=seq.shape[0]
    h=net.init_hidden(batch_size)
    seq=seq.cuda()
    out,h=net(seq,h)
    print(out.item())
    pred=round(out.item())
    if(pred==1.0):
        print("Positive")
    else:
        print("Negative")

In [32]:
predict_sentiment(best_trained_model,inputs)

0.822012722492218
Positive


In [33]:
test_review_neg = "This movie had terrible actors, I didn't like it at all."
inputs1=prepare_for_network(test_review_neg,200)

In [34]:
predict_sentiment(best_trained_model,inputs1)

0.022622698917984962
Negative
